#◢ DeOldify - Colorize your own photos!

####**Credits:**

Special thanks to:

Matt Robinson and María Benavente for pioneering the DeOldify image colab notebook.  

Dana Kelley for doing things, breaking stuff & having an opinion on everything.



---


#◢ Verify Correct Runtime Settings

**<font color='#FF000'> IMPORTANT </font>**

In the "Runtime" menu for the notebook window, select "Change runtime type." Ensure that the following are selected:
* Runtime Type = Python 3
* Hardware Accelerator = GPU 


In [0]:
from os import path
import torch

#◢ Git clone and install DeOldify

In [2]:
!git clone https://github.com/jantic/DeOldify.git DeOldify 

Cloning into 'DeOldify'...
remote: Enumerating objects: 1771, done.
remote: Total 1771 (delta 0), reused 0 (delta 0), pack-reused 1771
Receiving objects: 100% (1771/1771), 69.20 MiB | 29.04 MiB/s, done.
Resolving deltas: 100% (750/750), done.


In [3]:
cd DeOldify

/content/DeOldify


#◢ Setup

In [4]:
!pip install -r requirements.txt

     |████████████████████████████████| 215kB 6.7MB/s 
     |████████████████████████████████| 133kB 43.1MB/s 
     |████████████████████████████████| 1.8MB 39.6MB/s 
     |████████████████████████████████| 17.4MB 39.8MB/s 
     |████████████████████████████████| 61kB 24.8MB/s 
  Created wheel for ffmpeg: filename=ffmpeg-1.4-cp36-none-any.whl size=6084 sha256=f9749dbf9622d337d92e9c286cfd093c4328907160aa4e4dd4816bb05e7eabb2
  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built ffmpeg
  Found existing installation: fastai 1.0.55
    Uninstalling fastai-1.0.55:
      Successfully uninstalled fastai-1.0.55
  Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0


In [5]:
import fastai
from fasterai.visualize import *
from pathlib import Path
torch.backends.cudnn.benchmark=True

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 553507836/553507836 [00:07<00:00, 71841740.18it/s]


In [28]:
!mkdir 'models'
!wget wget https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0 -O ./models/ColorizeArtistic_gen.pth  
!wget wget https://www.dropbox.com/s/mwjep3vyqk5mkjc/ColorizeStable_gen.pth?dl=0 -O ./models/ColorizeStable_gen.pth

mkdir: cannot create directory ‘models’: File exists
--2019-08-06 05:52:59--  http://wget/
Resolving wget (wget)... failed: Name or service not known.
wget: unable to resolve host address ‘wget’
--2019-08-06 05:52:59--  https://www.dropbox.com/s/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6021:1::a27d:4101
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth [following]
--2019-08-06 05:53:00--  https://www.dropbox.com/s/raw/zkehq1uwahhbc2o/ColorizeArtistic_gen.pth
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb2489b8cf8c8db995fad7e8a6d.dl.dropboxusercontent.com/cd/0/inline/AmFK9SV49vHDcrL-Gt9xUaQE5-bB4vNFxBxz8qAI0Cq3P0uGw5a_5a-P27Qkh6TJ-rYni24qsvWKDLVJ1e2fVTYk2eHAJLyQnotWQTORSa0mMQ/file# [followi

In [41]:
colorizer = get_image_colorizer(artistic=False) #True/False

/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:442: UserWarning: Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.
  warn("Your training set is empty. If this is by design, pass `ignore_empty=True` to remove this warning.")
/usr/local/lib/python3.6/dist-packages/fastai/data_block.py:445: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


#◢ Instructions

### source_url
Type in a url to a direct link of an image.  Usually that means they'll end in .png, .jpg, etc. NOTE: If you want to use your own image, upload it first to a site like Imgur. 

### render_factor
The default value of 35 has been carefully chosen and should work -ok- for most scenarios (but probably won't be the -best-). This determines resolution at which the color portion of the image is rendered. Lower resolution will render faster, and colors also tend to look more vibrant. Older and lower quality images in particular will generally benefit by lowering the render factor. Higher render factors are often better for higher quality images, but the colors may get slightly washed out. 

#### How to Download a Copy
Simply right click on the displayed image and click "Save image as..."!

## Pro Tips
1. You can evaluate how well the image is rendered at each render_factor by using the code at the bottom (that cell under "See how well render_factor values perform on a frame here"). 
2. Keep in mind again that you can go up top and set artistic to False for the colorizer to use the 'Stable' model instead.  This will often tend to do better on portraits, and natural landscapes.  

## Troubleshooting
If you get a 'CUDA out of memory' error, you probably have the render_factor too high.  The max is 45 on 11GB video cards.

#◢ Colorize!!

In [44]:
source_url = 'https://i.imgur.com/qBoFzWb.jpg' #@param {type:"string"}
render_factor = 15  #@param {type: "slider", min: 7, max: 45}

if source_url is not None and source_url !='':
    image_path = colorizer.plot_transformed_image_from_url(url=source_url, render_factor=render_factor, compare=True)
    show_image_in_notebook(image_path)
else:
    print('Provide an image url and try again.')

Output hidden; open in https://colab.research.google.com to view.

## See how well render_factor values perform on the image here

In [43]:
for i in range(10,46,2):
    colorizer.plot_transformed_image('test_images/image.png', render_factor=i, display_render_factor=True, figsize=(8,8))

Output hidden; open in https://colab.research.google.com to view.

---
#⚙ Recommended image sources 
* [/r/TheWayWeWere](https://www.reddit.com/r/TheWayWeWere/)